# 🔍 Fun with Keyword Search using VideoDB

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Keyword_Search_1`.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 💬 Overview
--- 


In this tutorial, let’s explore the powerful functionality of Keyword Search in VideoDB. This feature enables users to efficiently locate any keyword or phrase within their video assets, streamlining the process of content discovery.

![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Keyword_Search_1.png)

![](https://raw.githubusercontent.com/video-db/videodb-cookbook/main/images/Intro_Outro_Inline/image.png)

## Setup
---

### 📦  Installing packages 

In [2]:
%pip install videodb


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key. 

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [3]:
import os

os.environ["VIDEO_DB_API_KEY"] = ""

## Steps
---

### 🌐 Step 1: Connect to VideoDB
Begin by establishing a connection to VideoDB using your API key

In [4]:
from videodb import connect

# Connect to VideoDB using your API key
conn = connect()
coll = conn.get_collection()

### 🎥 Step 2: Upload Videos

Upload the video to your VideoDB collection. You can upload the video asset from your local device or from a YouTube URL to upload the video from its source. This works as the base video for all the Keyword Search queries.

In [5]:
video = coll.upload(url="https://www.youtube.com/watch?v=Uvufun6xer8")
video.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/5205a412-3574-403f-b1cc-2be5479d872a.m3u8'

>You can upload from your local file system too by passing `file_path` in `upload()` 

### 🔊  Step 3: Index Spoken Words

Index the spoken words in your video to enable  keyword search.


In [6]:
video.index_spoken_words()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.52it/s]


### 🔎  Step 4: Search for any keyword

Utilize the keyword search by using `Video.search()` method with following parameters.

* pass search query in `query` parameter
* pass `SearchType.keyword` in `search_type`

In [7]:
from videodb import SearchType

results = video.search(query='metaverse', search_type=SearchType.keyword)

### 👀 Step 5: Preview and Share

Preview your video with a compilation of all the clips matching your search query.  You can access the stream link alongside the preview to share the Keyword Search result with others.

In [8]:
results.play()

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/5e946b7d-e4f8-431c-913d-bc0315502a75.m3u8'

## 🔎 Bonus : Refining Keyword Search results by adding padding.

Some keyword search results/ compilations may appear slightly choppy, or the cuts may feel abrupt. We can solve this issue by using VideoDB’s padding controls. Here’s how it works:


![](https://raw.githubusercontent.com/video-db/videodb-cookbook-assets/main/images/examples/Keyword_Search_2.png)

The resulting shots can be made smoother by including a little more context from before and after the matching timestamps. That’s exactly what padding controls enable: 

1. Create a timeline, just like we did previously, by using the  `Timeline()` function.
2. Create VideoAsset of each result shot using VideoAsset() with following parameters
    * `asset_id` : Video ID of the Video to which result belongs to [base video]
    * `start` : To adjust the beginning of the resulting shot, we subtract the padding duration from the original start timestamp. This ensures the resulting shot starts earlier in relation to the base video. 
    * `end`:  Likewise, we extend the resulting shot's end timestamp by adding the padding duration. This gives the resulting shot additional context from the base video.

Finally, we will add these freshly created assets in our Timeline by using `Timeline.add_inline()`

In [11]:
from videodb import play_stream
from videodb.timeline import Timeline
from videodb.asset import VideoAsset

timeline = Timeline(conn)

# Add padding
padding = 0.4

# Compile Video 
for shot in results.shots:
    asset = VideoAsset(
       asset_id=shot.video_id, start=shot.start-padding, end=shot.end+padding
    )
    timeline.add_inline(asset)

stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/57c576c9-8a00-4952-bdf3-323fdab7ed37.m3u8'

## 🎉 Conclusion
---
Keyword Search in VideoDB empowers users to extract valuable insights from their video assets with ease. For more information and advanced features, explore the [VideoDB Documentation](https://docs.videodb.io/) and join the VideoDB community on [GitHub](https://github.com/video-db) or [Discord](https://discord.com/invite/py9P639jGz) for support and collaboration.
